<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

In [4]:
import folium
import wget
import pandas as pd

In [42]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon
print("All libraries have been imported.")

All libraries have been imported.


If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [10]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [................................................................................] 7710 / 7710

Now, you can take a look at what are the coordinates for each site.


In [80]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [81]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [82]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [83]:
import pandas as pd
import folium
from folium import DivIcon
import wget

# Download and read the dataset
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df = pd.read_csv(spacex_csv_file)

# Clean the column headers
spacex_df.columns = spacex_df.columns.str.strip()  # Remove any leading/trailing spaces in column names

# Select relevant sub-columns: Launch Site, Lat, Long, and class
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Group by Launch Site and select the first entry for each site
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()

# Start location: NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Create a map centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add circles and markers for each launch site
for _, row in launch_sites_df.iterrows():
    # Coordinates for the launch site
    site_coordinate = [row['Lat'], row['Long']]
    
    # Add a circle for the launch site
    folium.Circle(
        location=site_coordinate,
        radius=1000,  # Radius in meters
        color='#d35400',  # Circle border color
        fill=True,
        fill_color='#ffa500',
        fill_opacity=0.5
    ).add_child(folium.Popup(f"{row['Launch Site']}")).add_to(site_map)
    
    # Add a marker for the launch site with a text label
    folium.Marker(
        location=site_coordinate,
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

# Display the map
site_map


100% [................................................................................] 7710 / 7710

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [84]:
import pandas as pd
import wget

# Load the dataset from the provided URL
spacex_csv_file = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(spacex_csv_file)

# Clean the column headers
spacex_df.columns = spacex_df.columns.str.strip()  # Remove any leading/trailing spaces in column names

# Select relevant sub-columns: Launch Site, Lat, Long, and class
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Group by Launch Site and select the first entry for each site
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()

# Function to check proximity to the Equator (latitudes near 0)
def proximity_to_equator(lat):
    return "Yes" if abs(lat) < 10 else "No"

# Function to check proximity to the coast
def proximity_to_coast(lon):
    # A basic check to see if the site is on the coast based on longitude.
    # Here, we're assuming that sites with longitudes near the U.S. coast are coastal.
    if -125 < lon < -66:  # General U.S. coast range for longitude
        return "Yes"
    return "No"

# Apply the functions to create new columns
launch_sites_df['Proximity to Equator'] = launch_sites_df['Lat'].apply(proximity_to_equator)
launch_sites_df['Proximity to Coast'] = launch_sites_df['Long'].apply(proximity_to_coast)

# Display the table as a single line
table = launch_sites_df.to_string(index=False, header=True)
print(table)


 Launch Site       Lat        Long  class Proximity to Equator Proximity to Coast
 CCAFS LC-40 28.562302  -80.577356      0                   No                Yes
CCAFS SLC-40 28.563197  -80.576820      1                   No                Yes
  KSC LC-39A 28.573255  -80.646895      1                   No                Yes
 VAFB SLC-4E 34.632834 -120.610745      0                   No                Yes


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [85]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [86]:
marker_cluster = MarkerCluster()


In [87]:
import pandas as pd
import folium
from folium import DivIcon
from folium.plugins import MarkerCluster
import wget

# Load the dataset from the provided URL
spacex_csv_file = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(spacex_csv_file)

# Clean the column headers
spacex_df.columns = spacex_df.columns.str.strip()  # Remove any leading/trailing spaces in column names

# Select relevant sub-columns: Launch Site, Lat, Long, and class
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Start location: NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Initialize the map with MarkerCluster
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers for all launch records
for _, row in spacex_df.iterrows():
    # Coordinates for the launch site
    site_coordinate = [row['Lat'], row['Long']]
    
    # Set marker color based on launch outcome (green for success, red for failure)
    marker_color = 'green' if row['class'] == 1 else 'red'
    
    # Add a marker for each launch record to the MarkerCluster
    folium.Marker(
        location=site_coordinate,
        icon=folium.Icon(color=marker_color),  # Set marker color based on outcome
        popup=f"{row['Launch Site']} - {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(marker_cluster)

# Display the map
site_map


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [89]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [90]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [91]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Load the dataset from the provided URL
spacex_csv_file = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(spacex_csv_file)

# Clean the column headers
spacex_df.columns = spacex_df.columns.str.strip()  # Remove any leading/trailing spaces in column names

# Select relevant sub-columns: Launch Site, Lat, Long, and class
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Calculate success rate for each launch site
launch_site_success_rate = spacex_df.groupby('Launch Site').agg(
    total_launches=('class', 'size'),
    successful_launches=('class', lambda x: (x == 1).sum())
)

# Calculate success rate as a percentage
launch_site_success_rate['success_rate'] = (launch_site_success_rate['successful_launches'] / launch_site_success_rate['total_launches']) * 100

# Start location: NASA Johnson Space Center (used as map center)
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Initialize the map with MarkerCluster
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
marker_cluster = MarkerCluster().add_to(site_map)

# Function to assign marker color based on the launch outcome (green for success, red for failure)
def get_marker_color(class_value):
    return 'green' if class_value == 1 else 'red'

# For each row in spacex_df, create a Marker object and customize the Marker icon property
for index, record in spacex_df.iterrows():
    # Coordinates for the launch site
    site_coordinate = [record['Lat'], record['Long']]
    
    # Set marker color based on the launch outcome (success/failure)
    marker_color = get_marker_color(record['class'])
    
    # Create a marker for the launch site with the specified icon color (green for success, red for failure)
    marker = folium.Marker(
        location=site_coordinate,
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}",  # Popup shows outcome
        icon=folium.Icon(color='white', icon_color=marker_color)  # Icon color based on success/failure
    )
    
    # Add the marker to the marker cluster
    marker_cluster.add_child(marker)

# Add text labels for the success rate for each launch site (outside the markers)
for launch_site, data in launch_site_success_rate.iterrows():
    # Coordinates for the launch site (taking the first occurrence of lat/long)
    site_data = spacex_df[spacex_df['Launch Site'] == launch_site].iloc[0]
    site_coordinate = [site_data['Lat'], site_data['Long']]
    
    # Create a label with the success rate
    success_rate = round(data['success_rate'], 2)
    
    # Add a label with the success rate information using DivIcon (external label)
    folium.Marker(
        location=site_coordinate,
        icon=folium.DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color: black; font-weight: bold;">{launch_site}<br>Success Rate: {success_rate}%</div>'
        )
    ).add_to(site_map)

# Display the map
site_map


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [92]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=5,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

# Add MousePosition to the map
site_map.add_child(mouse_position)

# Display the map
site_map


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [93]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [94]:
# CCAFS LC-40 Launch Site Coordinates
launch_site_lat = 28.562302
launch_site_lon = -80.577356

In [95]:
# Coordinates of the closet COASTLINE
coastline_lat = 28.56321
coastline_lon = -80.56801
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f'Distance to Coastline: {round(distance_coastline,2)} km')

# Create and add a folium.Marker on the selected point on the map
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(100,-10),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold; width:200px;">Coastline {round(distance_coastline,2)} km</div>',
        )
    ).add_to(site_map)

Distance to Coastline: 0.92 km


In [96]:
#Coordinates of the closet RAIL LINE
rail_line_lat = 28.57209
rail_line_lon = -80.58520
distance_rail_line = calculate_distance(launch_site_lat, launch_site_lon, rail_line_lat, rail_line_lon)
print(f'Distance to Rail Line: {round(distance_rail_line,2)} km')

# Create and add a folium.Marker on the selected point on the map
distance_marker = folium.Marker(
    [rail_line_lat, rail_line_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(-40,-20),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold; width:200px;">Rail Line {round(distance_rail_line,2)} km</div>',
        )
    ).add_to(site_map)

Distance to Rail Line: 1.33 km


In [97]:
# Coordinates of a point on the ROAD circling the launch site
road_lat = 28.561695
road_lon = -80.575553
distance_road = calculate_distance(launch_site_lat, launch_site_lon, road_lat, road_lon)
print(f'Distance to Road: {round(distance_road,2)} km')

# Create and add a folium.Marker on the selected point on the map
distance_marker = folium.Marker(
    [road_lat, road_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(10,0),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold; width:160px;">Perimeter Road {round(distance_road,2)} km</div>',
        )
    ).add_to(site_map)

Distance to Road: 0.19 km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [98]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [99]:
import folium
from folium import DivIcon

# Coordinates for the launch site (CCAFS LC-40)
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Coordinates of the coastline, rail line, and road
coastline_lat = 28.56321
coastline_lon = -80.56801

rail_line_lat = 28.57209
rail_line_lon = -80.58520

road_lat = 28.561695
road_lon = -80.575553

# Pre-calculated distances (provided by you)
distance_coastline = 0.92  # km
distance_rail_line = 1.33  # km
distance_road = 0.19  # km

# Create the folium map centered at the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=13)

# Define the colors for the lines (blue for coastline, green for rail line, red for road)
coastline_color = 'blue'
rail_line_color = 'green'
road_color = 'red'

# Create PolyLine objects using the coordinates
coastline_coordinates = [(launch_site_lat, launch_site_lon), (coastline_lat, coastline_lon)]
rail_line_coordinates = [(launch_site_lat, launch_site_lon), (rail_line_lat, rail_line_lon)]
road_coordinates = [(launch_site_lat, launch_site_lon), (road_lat, road_lon)]

# Create the PolyLine and add it to the map with the specified colors
folium.PolyLine(locations=coastline_coordinates, weight=2, color=coastline_color).add_to(site_map)
folium.PolyLine(locations=rail_line_coordinates, weight=2, color=rail_line_color).add_to(site_map)
folium.PolyLine(locations=road_coordinates, weight=2, color=road_color).add_to(site_map)

# Add markers at the points (coastline, rail line, and road) with the names and pre-calculated distances
folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold;">Coastline\n{round(distance_coastline, 2)} km</div>',
    )
).add_to(site_map)

folium.Marker(
    [rail_line_lat, rail_line_lon],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold;">Rail Line\n{round(distance_rail_line, 2)} km</div>',
    )
).add_to(site_map)

folium.Marker(
    [road_lat, road_lon],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold;">Road\n{round(distance_road, 2)} km</div>',
    )
).add_to(site_map)

# Add a **yellow circle** around the launch site (only the launch site has a circle)
folium.Circle(
    location=[launch_site_lat, launch_site_lon],
    radius=500,  # Circle radius in meters
    color='yellow',
    fill=True,
    fill_opacity=0.5
).add_to(site_map)

# Add a marker for the launch site with the name label
folium.Marker(
    [launch_site_lat, launch_site_lon],
    icon=DivIcon(icon_size=(150, 36), icon_anchor=(0, 0), html='<div style="font-size:12px; color:#222; font-weight:bold;">CCAFS LC-40</div>')
).add_to(site_map)

# Display the map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


In [100]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the SpaceX launch data into a pandas dataframe from the URL
spacex_csv_file = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(spacex_csv_file)

# Get max and min payload
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    html.Br(),

    # Dropdown for Launch Site Selection
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
        ],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),

    # Pie Chart
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    # Payload Range Slider
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
        value=[min_payload, max_payload]
    ),
    html.Br(),

    # Scatter Chart
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# Callback for Pie Chart
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site',
                     title='Total Success Launches by Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        success_count = filtered_df['class'].sum()
        failure_count = len(filtered_df) - success_count
        data = pd.DataFrame({'Outcome': ['Success', 'Failure'],
                             'Count': [success_count, failure_count]})
        fig = px.pie(data, values='Count', names='Outcome',
                     title=f'Success vs Failure for {entered_site}')
        return fig

# Callback for Scatter Chart
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_scatter_plot(entered_site, payload_range):
    min_value, max_value = payload_range
    if entered_site == 'ALL':
        filtered_df = spacex_df[
            (spacex_df['Payload Mass (kg)'] >= min_value) &
            (spacex_df['Payload Mass (kg)'] <= max_value)
        ]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                         color='Booster Version', title='Correlation between Payload and Success for All Sites')
        return fig
    else:
        filtered_df = spacex_df[
            (spacex_df['Launch Site'] == entered_site) &
            (spacex_df['Payload Mass (kg)'] >= min_value) &
            (spacex_df['Payload Mass (kg)'] <= max_value)
        ]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                         color='Booster Version', title=f'Correlation between Payload and Success for {entered_site}')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
